# Web Scraping from Indeed Website

* Log: 02082017 Jiewei Chen Da Xue
* Log: 02102016 Jiewei Chen Da Xue Nick Ulle
* Log: 02212017 Jiewei Chen

## Step 1. Using API get 5000 Posts

In [22]:
import requests
import requests_cache
requests_cache.install_cache("cache")
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
publisher_id = "254802918413674"

http://api.indeed.com/ads/apisearch?publisher=254802918413674&q=java&l=austin%2C+tx&sort=&radius=&st=&jt=&start=&limit=&fromage=&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2

In [50]:
url = "http://api.indeed.com/ads/apisearch?publisher=254802918413674&format=json&q=data&l=sunnyvale%2C+ca&sort=&radius=0&st=&jt=internship&start=&limit=1000&fromage=30&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2"

In [3]:
def get_indeed(city, state, keyword, radius, jobtype = "internship", js = "json", start = 0, publisher_id = publisher_id):
    url = "http://api.indeed.com/ads/apisearch?"
    if city == "":
        location = state
    else: 
        location = city + ', ' + state
    result = requests.get(url, params = {
            "q": keyword,
            "v": 2,
            "format": js,
            "publisher": publisher_id,
            "l": location,
            "jt": jobtype, # "fulltime", "parttime", "contract", "internship", "temporary"
            "radius": radius,
            "start": start,
            "limit": 100})
    result = result.json()['results']
    return result

In [14]:
job_list = pd.DataFrame()
for i in range(0,5001,25):
    temp = pd.DataFrame(get_indeed('','ca', '"data analyst"', 0, start = i))
    job_list = job_list.append(temp)


In [15]:
print(job_list.shape)
job_list.head()

(4221, 18)


,city,company,country,date,expired,formattedLocation,formattedLocationFull,formattedRelativeTime,indeedApply,jobkey,jobtitle,onmousedown,snippet,source,sponsored,state,stations,url
0,Burbank,Kaiser Permanente,US,"Wed, 22 Feb 2017 00:08:47 GMT",False,"Burbank, CA","Burbank, CA",1 hour ago,False,fdbd1113c5011d69,Market Data Analyst - Undergraduate Internship,"indeed_clk(this,'5647');",Aptitude to perform <b>data</b> and process an...,Kaiser Permanente,False,CA,,http://www.indeed.com/viewjob?jk=fdbd1113c5011...
1,Oakland,Clorox,US,"Mon, 06 Feb 2017 23:25:16 GMT",False,"Oakland, CA","Oakland, CA 94612",15 days ago,False,456e90c7ee9c4936,Global Insights Data Analyst Data Science Inte...,"indeed_clk(this,'5647');",The Global Insights <b>Data</b> <b>Analyst</b>...,The Clorox Company,False,CA,,http://www.indeed.com/viewjob?jk=456e90c7ee9c4...
2,Redwood City,Proteus Digital Health,US,"Fri, 17 Feb 2017 04:08:45 GMT",False,"Redwood City, CA","Redwood City, CA 94065",4 days ago,False,676ec709d44d20b9,Data Analyst Intern,"indeed_clk(this,'5647');","Mine, clean, and categorize historical <b>data...",Proteus Digital Health,False,CA,,http://www.indeed.com/viewjob?jk=676ec709d44d2...
3,Santa Clara,GLOBALFOUNDRIES,US,"Wed, 08 Feb 2017 07:12:14 GMT",False,"Santa Clara, CA","Santa Clara, CA",13 days ago,False,2597a304d86609fc,Data Analyst-Intern,"indeed_clk(this,'5647');",<b>Data</b> analysis and validation. Database ...,GLOBALFOUNDRIES,False,CA,,http://www.indeed.com/viewjob?jk=2597a304d8660...
4,San Francisco,Samasource,US,"Fri, 17 Feb 2017 03:46:33 GMT",False,"San Francisco, CA","San Francisco, CA",4 days ago,False,e726041a4f3ec15b,Project/Data Analyst Internship,"indeed_clk(this,'5647');",The <b>analyst</b> intern analyzes project per...,Samasource,False,CA,,http://www.indeed.com/viewjob?jk=e726041a4f3ec...


## Problems

<ol>
    <li> Can only get at max 25 jobs per request <font color="red">Change start params</font>
    <li> When search for "Data Analyst", it is actually searching for "Data" and "Analyst" as two seperated key words
    <li> Cannot extract the job requirement information <font color="red"> Probably need web scraping </font>
    <li> 
</ol>

## Step 2. Webscraping each post

In [66]:
url_list = list(job_list['url'])

In [67]:
def get_post_details(url):
    """
    This function extracts information of the one post by taking a url as input
    Output is a dictionary with job title, company name, and description of the job.
    """
    post_doc = requests.get(url).content
    post = BeautifulSoup(post_doc, 'html.parser')
    
    title = post.find('b',attrs={'class':'jobtitle'}).text
    company = post.find(attrs={'class':'company'}).text
    dis_html = post.find_all(attrs={'id':'job_summary'})
    post_dict = {'title':title, 'company':company, 'description':dis_html}
    return post_dict

In [70]:
post_summary = pd.DataFrame()
for url in url_list:
    temp = pd.DataFrame(list(get_post_details(url).values())).T
    post_summary = post_summary.append(temp)
post_summary.columns = ['company','description','job title']    

In [71]:
post_summary.tail()

,company,description,job title
0,KLA-Tencor,"[<span class=""summary"" id=""job_summary"">KLA-Te...",Intern / Co-Op Technical
0,Lyft Corporate,"[<span class=""summary"" id=""job_summary"">Analyt...",Data Analyst Intern (Summer 2017)
0,Rubin Postaer and Associates (RPA),"[<span class=""summary"" id=""job_summary"">The Re...","Summer Internship - Research, Analytics + Insi..."
0,Quidel Corporation,"[<span class=""summary"" id=""job_summary"">Quidel...","Internship, Data Analyst/Business Intelligence..."
0,Teradata,"[<span class=""summary"" id=""job_summary"">Terada...",Data Analyst and Test Development Intern (Summ...


In [75]:
list(post_summary['description'])[0]

[<span class="summary" id="job_summary">Lay the groundwork for a dynamic future with an industry leader. At Kaiser Permanente, you'll forge lasting relationships with
 <br><br>
 colleagues across the organization while enjoying the opportunity to grow your experience and hone your expertise on assignments
 <br><br>
 that have real impact. Our interns have worked on everything from analyzing financial software and developing new marketing
 <br><br>
 strategies, to designing mobile apps and helping clinicians improve care delivery. We want you to get the most out of your time here
 <br><br>
 and will give you projects that challenge you to think freely, question thoroughly, and explore deeply.
 <br><br>
 <b>Essential Functions:</b><br>
 <ul>
 <li>Contributing to a designated project or initiative to meet a KP business objective.</li>
 <li>Report development and analytics.</li>
 <li>Project management - planning, execution, and measurement.</li>
 <li>Learning new applications needed to co